In [1]:
'''
HackerRank Trade Order Design Challenge

Order type:
    - market : trade at available price. discard remaining qty.
    - limit : trade at price limit. keep remaining order.
    - stop : trade only when future trade occurs in a price threshold.
    - cancel : cancel existing order.
    
Example:

<input>
orde# type buy/sell qty price

1 limit buy 10 99.00
2 limit buy 15 100.00
3 limit buy 3 100.50
4 limit sell 5 100.00
5 limit buy 5 99.50
6 stop sell 3 99.49
7 cancel na 2 0.00
8 market sell 6 0.00

<output>
match new_order# old_order# qty trade_price

match 4 3 3 100.50
match 4 2 2 100.00
match 8 5 5 99.50
match 8 1 1 99.00
match 6 1 3 99.00
'''

'\nHackerRank Trade Order Design Challenge\n\nOrder type:\n    - market : trade at available price. discard remaining qty.\n    - limit : trade at price limit. keep remaining order.\n    - stop : trade only when future trade occurs in a price threshold.\n    - cancel : cancel existing order.\n    \nExample:\n\n<input>\norde# type buy/sell qty price\n\n1 limit buy 10 99.00\n2 limit buy 15 100.00\n3 limit buy 3 100.50\n4 limit sell 5 100.00\n5 limit buy 5 99.50\n6 stop sell 3 99.49\n7 cancel na 2 0.00\n8 market sell 6 0.00\n\n<output>\nmatch new_order# old_order# qty trade_price\n\nmatch 4 3 3 100.50\nmatch 4 2 2 100.00\nmatch 8 5 5 99.50\nmatch 8 1 1 99.00\nmatch 6 1 3 99.00\n'

In [2]:
# command pattern with receiver=trade_system
class Order:
    def __init__(self, id, type, qty, price, trade_system):
        self.id = id
        self.type = type
        self.qty = qty
        self.price = price
        self.trade_system = trade_system
    
    def execute(self):
        pass
            
class StopOrder(Order):
    def execute(self):
        new_order = MarketOrder(self.id, self.type, self.qty, self.price, self.trade_system)
        stop_listener = StopListener(self.trade_system, new_order)
        self.trade_system.add_listener(stop_listener)
        
class CancelOrder(Order):
    def execute(self):
        self.trade_system.cancel_order(self.qty)
        
        
class TradeOrder(Order):
    def is_active(self):
        return (self.qty > 0)
    
    def cancel(self):
        self.qty = 0
    
    def match(self, order):
        pass
    
    def complete_trade(self):
        pass
    
    def trade(self, old_order):
        new_order = self
        qty = min(new_order.qty, old_order.qty)
        price = old_order.price
        
        new_order.qty -= qty
        old_order.qty -= qty
        
        print('match %d %d %d %.2f' %(new_order.id, old_order.id, qty, price))
        return qty, price
    

class LimitOrder(TradeOrder):
    def match(self, order):
        if self.type == 'buy':
            return (order.type != self.type) and (self.qty > 0 and order.qty > 0) and (order.price <= self.price)
        elif self.type == 'sell':
            return (order.type != self.type) and (self.qty > 0 and order.qty > 0) and (order.price >= self.price)
        return False
    
    def complete_trade(self):
        self.trade_system.add_order(self)
        
    def execute(self):
        self.trade_system.process_trade_order(self)

class MarketOrder(TradeOrder):
    def match(self, order):
        return (order.type != self.type) and (self.qty > 0 and order.qty > 0)
    
    def complete_trade(self):
        self.cancel()
    
    def execute(self):
        self.trade_system.process_trade_order(self)
        
        
# Factory pattern for creating Order                
class OrderFactory:
    def __init__(self, trade_system):
        self.trade_system = trade_system
        
    def create(self, id, line):
        data = [s.strip() for s in line.split(' ')]
        if len(data) != 4:
            return None
        
        command = data[0]
        type = data[1]
        qty = int(data[2])
        price = float(data[3])
        
        order = None
        if command == 'market':
            order = MarketOrder(id, type, qty, price, self.trade_system)
        
        elif command == 'limit':
            order = LimitOrder(id, type, qty, price, self.trade_system)
        
        elif command == "stop":
            order = StopOrder(id, type, qty, price, self.trade_system)
        
        elif command == "cancel":
            order = CancelOrder(id, type, qty, price, self.trade_system)
                  
        return order        
    
    
class TradeListener:
    def __init__(self, trade_system):
        self.trade_system = trade_system
    def update(self, qty, price):
        pass
    
class StopListener(TradeListener):
    def __init__(self, trade_system, order):
        self.trade_system = trade_system
        self.order = order
        
    def update(self, qty, price):
        if (self.order.type == 'sell' and price <= self.order.price) \
            or (self.order.type == 'buy' and price >= self.order.price) :
            self.order.execute()
            
            
# friend class of Order. 
# access to private element of Order class
# event-listener for stop-type order (invoked when price hits some level)
class TradeSystem:
    def __init__(self):
        # for performance, use heap/BST
        self.buy_list = []
        self.sell_list = []
        self.listener_list = []
    
    def process_trade_order(self, order):
        
        # for performance, use heap/BST
        # also iterate one by one instead of having entire list
        # we just code here for simplicity
        li = []
        if order.type == 'buy':
            li = sorted(self.sell_list, key=lambda x: [x.price, x.id])
        elif order.type == 'sell':
            li = sorted(self.buy_list, key=lambda x: [-x.price, x.id])
            
        trade_list = []
        for target in li:
            if order.match(target):
                qty, price = order.trade(target)
                trade_list.append((qty, price))
            if not order.is_active():
                break
        
        order.complete_trade()
        
        for qty, price in trade_list:
            self.notify_trade(qty, price)
        
    def add_order(self, order):
        if order.type == 'buy':
            self.buy_list.append(order)
        elif order.type == 'sell':
            self.sell_list.append(order)
    
    def add_listener(self, listener):
        self.listener_list.append(listener)
        
    def notify_trade(self, qty, price):
        for listener in self.listener_list:
            listener.update(qty, price)
    
    def cancel_order(self, id):
        self.buy_list = [x for x in self.buy_list if x.id != id]
        self.sell_list = [x for x in self.sell_list if x.id != id]
        self.listener_list = [x for x in self.listener_list if x.order.id != id]    
    
    def display(self):
        print('--buy--')
        for x in self.buy_list:
            print('id:%d, q:%d, price:%.2f' %(x.id, x.qty, x.price))
        print('--sell--')
        for x in self.sell_list:
            print('id:%d, q:%d, price:%.2f' %(x.id, x.qty, x.price))
        print('--listen--')
        for x in self.listener_list:
            print('id:%d, q:%d, price:%.2f' %(x.order.id, x.order.qty, x.order.price))

In [3]:
import sys
class Main:
    def run(self):
        trade_system = TradeSystem()
        order_factory = OrderFactory(trade_system)
        id = 0
        for line in open('input.txt', 'r'):
        #for line in sys.stdin:
            id += 1
            order = order_factory.create(id, line)
            if order:
                order.execute()        
        #trade_system.display()
Main().run()

match 4 3 3 100.50
match 4 2 2 100.00
match 8 5 5 99.50
match 8 1 1 99.00
match 6 1 3 99.00
